In [4]:
import os
import json
import glob
import duckdb
import pandas as pd
from pathlib import Path
import shutil
import re

# Configuration
MASTER_DB_PATH = "/mnt/data4/recombia.planter/master.cpy.duckdb"
SAMPLES_DIR = "/mnt/data4/recombia.planter/"
OUTPUT_DB_PATH = "/mnt/data4/recombia.planter/master.updated.duckdb"

# Get list of sample directories
sample_dirs = []
for item in os.listdir(SAMPLES_DIR):
    if item.startswith("SRR") and os.path.isdir(os.path.join(SAMPLES_DIR, item)):
        sample_dirs.append(item)

print(f"Found {len(sample_dirs)} sample directories")

# List first 5 sample dirs to confirm they look right
for i, sample in enumerate(sample_dirs[:5]):
    print(f"Sample {i+1}: {sample}")

# Create a copy of the master database to work with
shutil.copy(MASTER_DB_PATH, OUTPUT_DB_PATH)
print(f"Created copy of master database at {OUTPUT_DB_PATH}")

# Check schema of the master database
master_tables = []
with duckdb.connect(OUTPUT_DB_PATH) as con:
    tables = con.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'main'
    """).fetchall()
    master_tables = [t[0] for t in tables]

print("Tables in master database:")
for table in master_tables:
    print(f" - {table}")

# Create schema for expression and gene_protein_map tables if they don't exist
expression_schema = """
CREATE TABLE IF NOT EXISTS expression (
    gene_seqhash_id VARCHAR NOT NULL,
    sample_id VARCHAR NOT NULL,
    tpm DOUBLE NOT NULL,
    num_reads DOUBLE NOT NULL,
    effective_length DOUBLE NOT NULL,
    PRIMARY KEY (gene_seqhash_id, sample_id),
    FOREIGN KEY (sample_id) REFERENCES sra_metadata(sample_id)
);
"""

gene_protein_map_schema = """
CREATE TABLE IF NOT EXISTS gene_protein_map (
    gene_seqhash_id VARCHAR PRIMARY KEY,
    protein_seqhash_id VARCHAR NOT NULL,
    FOREIGN KEY (protein_seqhash_id) REFERENCES sequences(seqhash_id)
);
"""

with duckdb.connect(OUTPUT_DB_PATH) as con:
    # Create the tables if they don't exist
    if 'expression' not in master_tables:
        con.execute(expression_schema)
        print("Created expression table")

    if 'gene_protein_map' not in master_tables:
        con.execute(gene_protein_map_schema)
        print("Created gene_protein_map table")

# Function to parse gene-protein relationships from a TransDecoder .pep file
def parse_gene_protein_map(pep_file):
    """
    Extract gene-to-protein mapping from TransDecoder .pep file
    Returns a list of (gene_id, protein_id) tuples
    """
    gene_protein_map = []

    with open(pep_file, 'r') as f:
        for line in f:
            if line.startswith('>'):
                # Example header:
                # >v1_DLS_1234567890abcdef1234567890abcdef1234567890abcdef1234567890abcdef.p1
                # The gene ID is everything before the .p1
                header = line.strip()[1:]  # Remove '>'
                parts = header.split()
                protein_id = parts[0]  # Full protein ID with .p1

                # Extract gene ID by removing the .p suffix
                gene_id = re.sub(r'\.p\d+$', '', protein_id)

                gene_protein_map.append((gene_id, protein_id))

    return gene_protein_map

# Function to parse expression data from quant.json file
def parse_expression_data(quant_json, sample_id):
    """
    Extract expression data from quant.json file
    Returns a list of dictionaries with expression data
    """
    with open(quant_json, 'r') as f:
        data = json.load(f)

    # Add sample_id to each record
    for record in data:
        record['sample'] = sample_id

    return data

# Process each sample
for i, sample_id in enumerate(sample_dirs):
    print(f"Processing sample {i+1}/{len(sample_dirs)}: {sample_id}...")

    try:
        sample_dir = os.path.join(SAMPLES_DIR, sample_id)
        sample_db_path = os.path.join(sample_dir, f"{sample_id}.duckdb")

        # Check if this sample exists in the master database
        with duckdb.connect(OUTPUT_DB_PATH) as con:
            result = con.execute("""
                SELECT COUNT(*) FROM sra_metadata WHERE sample_id = ?
            """, [sample_id]).fetchone()

            if result[0] == 0:
                print(f"  - Sample not found in master database, skipping")
                continue

        # Process expression data
        quant_json_path = os.path.join(sample_dir, "quants", f"{sample_id}.quant.json")
        if os.path.exists(quant_json_path):
            print(f"  - Found quant.json at {quant_json_path}")

            # Parse expression data
            try:
                expression_data = parse_expression_data(quant_json_path, sample_id)
                print(f"  - Parsed {len(expression_data)} expression records")

                # Create expression records
                expression_records = []
                for record in expression_data:
                    expression_records.append({
                        'gene_seqhash_id': record['Name'],  # This is the gene ID (without .p suffix)
                        'sample_id': record['sample'],
                        'tpm': record['TPM'],
                        'num_reads': record['NumReads'],
                        'effective_length': record['EffectiveLength']
                    })

                # Insert into master database
                with duckdb.connect(OUTPUT_DB_PATH) as con:
                    # Convert to DataFrame for easier insertion
                    expr_df = pd.DataFrame(expression_records)

                    # Insert using execute_many
                    con.execute("""
                        INSERT OR IGNORE INTO expression
                        (gene_seqhash_id, sample_id, tpm, num_reads, effective_length)
                        VALUES (?, ?, ?, ?, ?)
                    """, expr_df[['gene_seqhash_id', 'sample_id', 'tpm', 'num_reads',
'effective_length']].values.tolist())

                    print(f"  - Inserted expression data for {sample_id}")
            except Exception as e:
                print(f"  - Error processing expression data: {e}")
        else:
            print(f"  - No quant.json found")

        # Process gene-protein mapping
        pep_file_path = os.path.join(sample_dir, "transdecoder", f"{sample_id}.pep")
        if os.path.exists(pep_file_path):
            print(f"  - Found .pep file at {pep_file_path}")

            # Parse gene-protein map
            try:
                gene_protein_map = parse_gene_protein_map(pep_file_path)
                print(f"  - Parsed {len(gene_protein_map)} gene-protein mappings")

                # Insert into master database
                with duckdb.connect(OUTPUT_DB_PATH) as con:
                    # Insert using execute_many
                    con.executemany("""
                        INSERT OR IGNORE INTO gene_protein_map
                        (gene_seqhash_id, protein_seqhash_id)
                        VALUES (?, ?)
                    """, gene_protein_map)

                    print(f"  - Inserted gene-protein mapping for {sample_id}")
            except Exception as e:
                print(f"  - Error processing gene-protein mapping: {e}")
        else:
            print(f"  - No .pep file found")

        print(f"  - Done processing sample {sample_id}")
    except Exception as e:
        print(f"  - Error processing sample {sample_id}: {e}")

# Verify the data in the updated master database
with duckdb.connect(OUTPUT_DB_PATH) as con:
    # Check expression table
    count = con.execute("SELECT COUNT(*) FROM expression").fetchone()[0]
    print(f"Expression table contains {count} records")

    # Sample of expression data
    if count > 0:
        print("Sample of expression data:")
        sample = con.execute("""
            SELECT gene_seqhash_id, sample_id, tpm, num_reads, effective_length
            FROM expression
            LIMIT 5
        """).fetchall()
        for row in sample:
            print(f"  {row}")

    # Check gene_protein_map table
    count = con.execute("SELECT COUNT(*) FROM gene_protein_map").fetchone()[0]
    print(f"gene_protein_map table contains {count} records")

    # Sample of gene_protein_map data
    if count > 0:
        print("Sample of gene_protein_map data:")
        sample = con.execute("""
            SELECT gene_seqhash_id, protein_seqhash_id
            FROM gene_protein_map
            LIMIT 5
        """).fetchall()
        for row in sample:
            print(f"  {row}")

    # Check for any referential integrity issues
    print("Checking for referential integrity issues...")

    # Check gene_protein_map references
    missing = con.execute("""
        SELECT COUNT(*)
        FROM gene_protein_map gpm
        LEFT JOIN sequences s ON gpm.protein_seqhash_id = s.seqhash_id
        WHERE s.seqhash_id IS NULL
    """).fetchone()[0]

    print(f"Gene-protein map entries with missing sequence references: {missing}")

    # Check expression references
    missing = con.execute("""
        SELECT COUNT(*)
        FROM expression e
        LEFT JOIN sra_metadata sm ON e.sample_id = sm.sample_id
        WHERE sm.sample_id IS NULL
    """).fetchone()[0]

    print(f"Expression entries with missing sample references: {missing}")

print(f"Database update complete. Updated database saved to {OUTPUT_DB_PATH}")

Found 117 sample directories
Sample 1: SRR18070787
Sample 2: SRR12068552
Sample 3: SRR8053131
Sample 4: SRR19034773
Sample 5: SRR10444682
Created copy of master database at /mnt/data4/recombia.planter/master.updated.duckdb
Tables in master database:
 - annotations
 - clusters
 - cluster_members
 - ec_numbers
 - go_terms
 - kegg_info
 - schema_version
 - sequences
 - sra_metadata
Created expression table
Created gene_protein_map table
Processing sample 1/117: SRR18070787...
  - Found quant.json at /mnt/data4/recombia.planter/SRR18070787/quants/SRR18070787.quant.json
  - Parsed 17331 expression records
  - Error processing expression data: Invalid Input Error: Prepared statement needs 5 parameters, 17331 given
  - Found .pep file at /mnt/data4/recombia.planter/SRR18070787/transdecoder/SRR18070787.pep
  - Parsed 21436 gene-protein mappings
  - Inserted gene-protein mapping for SRR18070787
  - Done processing sample SRR18070787
Processing sample 2/117: SRR12068552...
  - Found quant.json a

KeyboardInterrupt: 